# New Sprint Doc flow

The following notebook covers the steps to copy a confluence doc, rename it and then send out some slack messages.

In [ ]:
from prefect import flow, task, get_run_logger
from atlassian import Confluence
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from datetime import datetime, timezone, timedelta
import os
from dotenv import load_dotenv
from dataclasses import dataclass

load_dotenv()

In [ ]:
@task
def set_variables():
    # Use UTC date for consistency; adjust if you prefer local time
    today = datetime.now(timezone.utc).date()
    end_date = today + timedelta(days=13)
    new_doc_title = f"{today.isoformat()} - {end_date.isoformat()}"

    return {
        "atlassian_key": os.getenv("ATLASSIAN_KEY", ""),
        "atlassian_domain": os.getenv("ATLASSIAN_DOMAIN", ""),
        "atlassian_email": os.getenv("ATLASSIAN_EMAIL", ""),
        "slack_key": os.getenv("SLACK_KEY", ""),
        "confluence_copy_target_id": os.getenv("CONFLUENCE_COPY_TARGET_ID", ""),
        "slack_user_id": os.getenv("SLACK_USER_ID", ""),
        "confluence_space_id": os.getenv("CONFLUENCE_SPACE_ID", ""),
        "new_doc_title": new_doc_title,
        "template_id": os.getenv("CONFLUENCE_COPY_TARGET_ID", "")
    }

In [ ]:
@task
def copy_template(variables: dict):
    confluence: Confluence = Confluence(
        url=variables["atlassian_domain"],
        username=variables["atlassian_email"],
        password=variables["atlassian_key"],
        cloud=True,
    )
    source_page = confluence.get_page_by_id(page_id=variables["confluence_copy_target_id"], expand="body.storage,ancestors")
    
    existing = confluence.get_page_by_title(space=variables["confluence_space_id"], title=variables["new_doc_title"])
    if existing:
        return {
            "doc_id": existing["id"],
            "title": existing["title"],
            "created": False
        }

    body_html = source_page["body"]["storage"]["value"]
    parent_id = None
    ancestors = source_page.get("ancestors") or []
    if ancestors:
        parent_id = ancestors[-1]["id"]

    new_page = confluence.create_page(
        space=variables["confluence_space_id"],
        title=variables["new_doc_title"],
        body=body_html,
        parent_id=parent_id
    )

    return {
        "doc_id": new_page["id"],
        "title": new_page["title"],
        "created": True
    }

In [ ]:
copy_template(set_variables())

In [ ]:
@task
def send_slack_update(doc_id: str, variables: dict):
    token = variables.get("slack_key")
    user_id = variables.get("slack_user_id")
    spaces_key = variables.get("confluence_space_id")

    client = WebClient(token=token)
    try:
        im = client.conversations_open(users=user_id)  # single user id
        channel_id = im["channel"]["id"]
        client.chat_postMessage(
            channel=channel_id,
            text=f"""@channel
Reminder: please enter your success challenges for the current sprint. 
Enter them here: https://bitovi.atlassian.net/wiki/spaces/{spaces_key}/pages/{doc_id}
If you need help or have questions, please ask in this thread or DM me.)"""
        )
    except SlackApiError as e:
        print(f"Slack error: {e.response.get('error','unknown_error')}")

### just a test on sending the slack message

In [ ]:
send_slack_update(set_variables().get("confluence_copy_target_id"), set_variables())

In [ ]:
@task
def send_congrats(variables: dict):
    token = variables.get("slack_key")
    user_id = variables.get("slack_user_id")

    client = WebClient(token=token)
    try:
        im = client.conversations_open(users=user_id)  # single user id
        channel_id = im["channel"]["id"]
        client.chat_postMessage(
            channel=channel_id,
            text=f"""Congrats, your confluence page has been created and shared on Slack! 🎉"""
        )
    except SlackApiError as e:
        print(f"Slack error: {e.response.get('error','unknown_error')}")

# Main Flow

In [ ]:
@flow
def sprint_success_flow():
    vars = set_variables()
    doc = copy_template(vars)
    send_slack_update(doc.get("doc_id"), vars)
    send_congrats(vars)

In [ ]:
# Biweekly deployment (every 14 days) anchored to a Thursday noon UTC
anchor = datetime(2025, 9, 25, 12, 0, tzinfo=timezone.utc)

sprint_success_flow.deploy(
    name="sprint-scheduler",
    interval=timedelta(days=14),
    anchor_date=anchor,
    tags=["sprint", "biweekly"],
)

In [ ]:
sprint_success_flow()

Do the following for deployments

```bash
jupyter nbconvert --to script nbs/02_new_sprint_doc.ipynb
```

```bash
prefect deployment build my_flows.py:sprint_success_flow \
    -n "sprint-scheduler" \
    --cron "0 12 * * 4"
```

```bash
prefect deployment apply sprint_success_flow-deployment.yaml
```